In [4]:
import pandas as pd
import numpy as np
import json
import os
import requests
from datetime import datetime
import time

In [243]:
API_KEY = 'VJga0fAn'
blacklist = []
retry_set = set()
STATIONS = pd.read_csv('data/stations.csv')

In [244]:
STATIONS

,Weather Station ID,Country,Name,Latitude,Longitude,Elevation,ICAO ID,WMO ID,IATA ID,Time Zone
0,PABL0,US,Buckland Airport,65.9667,-161.1500,7.0,PABL,NaN,BKC,NaN
1,1001,NO,Jan Mayen,70.9333,-8.6667,10.0,ENJA,1001.0,NaN,Europe/Oslo
2,1002,NO,Grahuken,79.7833,14.4667,NaN,NaN,1002.0,NaN,Europe/Oslo
3,1003,NO,Hornsund,77.0000,15.5000,10.0,NaN,1003.0,NaN,Europe/Oslo
4,1004,NO,Ny-Alesund Ii,78.9167,11.9333,8.0,ENAS,1004.0,NaN,Europe/Oslo
...,...,...,...,...,...,...,...,...,...,...
30404,D5851,DE,Herdorf,50.7668,7.9735,483.0,NaN,NaN,NaN,Europe/Berlin
30405,D5930,DE,Garmisch-Partenkirchen (Kanker),47.4921,11.1747,920.0,NaN,NaN,NaN,Europe/Berlin
30406,D5958,DE,Glees (Laacher See),50.3996,7.2651,287.0,NaN,NaN,NaN,Europe/Berlin
30407,D9041,DE,Zinnwald,50.7313,13.7529,871.0,NaN,NaN,NaN,Europe/Berlin


## Methods

In [295]:
def drop_null_vals(df,axis='both',subset=[]):
    '''
    Drops columns with all
    nan values from a given 
    data frame.
    
    Parameters
    ----------
    df : DataFrame
        DataFrame for which
        columns are to be
        dropped.
        
    axis : str
        Drops all rows with
        nan if axis=rows,
        all columns if axis=columns,
        and both if axis=both.
        
    subset : list of str
        For all columns in
        subset, remove the
        NaN rows.
    '''
    assert(isinstance(df,pd.DataFrame))
    assert(isinstance(axis,str))
    assert(isinstance(subset,list))
    assert(isinstance(col,str) for col in subset)
    
    df = df.dropna(subset=subset)
    
    if(axis=='rows'):
        df = df.dropna(how='all',axis=0)
    elif(axis=='columns'):
        df = df.dropna(how='all',axis=1)
    elif(axis=='both'):
        df = df.dropna(how='all',axis=0).dropna(how='all',axis=1)
    
    return df

def getWeatherData(latitude,longitude):
    '''
    Returns temperature 
    and humidity data 
    as per the latitude 
    and longitude entered.
    
    Parameters
    ----------
    latitude : float
        Latitude of region
        for fetching the
        weather data
        
    longitude : float
        Longitude of region
        for fetching weather
        data
    '''
    assert(isinstance(latitude,float))
    assert(isinstance(longitude,float))
    
    station = getNearbyStation(latitude,longitude)
    if(station is None):
        return
    else:
        print("Station found for (%f,%f):\n%s"%(latitude,longitude,station))
    url = createHourlyURL(station,START_DATE,END_DATE)
    weather_data = dict()
    print("URL for getting weather data for (%f,%f):\n%s"%(latitude,longitude,url))
    response = requests.get(url)
    code = response.status_code
    print("Got response, status = %f"%(code))
    try:
        df=pd.DataFrame(response.json()['data'])
        date_list = list(df['time_local'].unique())
#         print(retry_set)
        retry_set.discard((latitude,longitude))
        for date in date_list:
            mean_temp=df[df['time_local']==date]['temperature'].mean()
            mean_humidity=df[df['time_local']==date]['humidity'].mean()
            df_dict = dict()
            df_dict['temperature'] = mean_temp
            df_dict['humidity'] = mean_humidity
            weather_data[date] = df_dict
    except:
        print('No data found for (%f,%f)'%(latitude,longitude))
        print("Retry has to be done for (%f,%f)"%(latitude,longitude))
        coordinates = (latitude,longitude)
        retry_set.add(coordinates)
    
    return weather_data

def getNearbyStation(latitude,longitude):
    '''
    Given the latitude and
    longitude of a area,
    returns the nearest station
    to it.
    
    Parameters
    ----------
    latitude : float
        Latitude of region
        for fetching the
        weather data
        
    longitude : float
        Longitude of region
        for fetching weather
        data
    '''
    assert(isinstance(latitude,float))
    assert(isinstance(longitude,float))

    url = createStationURL(latitude,longitude)

    try:
        filter1 = stations['Latitude']==latitude
        filter2 = stations['Longitude']==longitude
        return stations.where(filter1 & filter2).dropna(how='all')['Weather Station ID'][0]
    except:
        try:
            print('Station does not exist in "Stations.csv", fetching from api...')
            response = requests.get(url)
            code = response.status_code
            result = response.json()['data'][0]['id']
            df = pd.DataFrame({"Weather Station ID":[result], "Latitude":[latitude], "Longitude":[longitude]})
            STATIONS.append(df, ignore_index=True)
            print('Station fetched for (%f,%f) is: %s'%(latitude,longitude,result))
            return result
        except Exception as ex:
            print('No station found for (%f,%f). URL: %s'%(latitude,longitude,url))
            print('exception: ',ex)
            if(code == 403):
                print("Retry has to be done for (%f,%f)"%(latitude,longitude))
                retry_set.add((latitude,longitude))
            return
        
def createStationURL(latitude,longitude):
    '''
    Returns station URL
    for given latitude
    and longitude.
    
    Parameters
    ----------
    latitude : float
        Latitude of region
        for fetching the
        weather data
        
    longitude : float
        Longitude of region
        for fetching weather
        data
    '''
    assert(isinstance(latitude,float))
    assert(isinstance(longitude,float))
    
    return 'https://api.meteostat.net/v1/stations/nearby?lat='+str(latitude)+'&lon='+str(longitude)+'&limit=1&key='+API_KEY

def createHourlyURL(station_id,start_date,end_date):
    '''
    Creates weather URL
    for given station,
    start date and end
    date.
    
    Parameters
    ----------
    station_id : str
        Station id of the
        region for which
        data is to be fetched
        
    start_date : str
        Date starting from which
        data is to be fetched.
        
    end_date : str
        Date ending at which
        data is to be fetched
    '''
    assert(isinstance(station_id,str))
    assert(isinstance(start_date,str))
    assert(isinstance(end_date,str))
    
    url = 'https://api.meteostat.net/v1/history/hourly?station='+station_id+'&start='+start_date+'&end='+end_date+'&time_zone=Europe/London&time_format=Y-m-d&key='+API_KEY

    return url

## Data

### 1. covid_19_data.csv

Sno - Serial number<br/>
ObservationDate - Date of the observation in MM/DD/YYYY<br/>
Province/State - Province or state of the observation (Could be empty when missing)<br/>
Country/Region - Country of observation<br/>
Last Update - Time in UTC at which the row is updated for the given province or country. (Not standardised and so please clean before using it)<br/>
Confirmed - Cumulative number of confirmed cases till that date<br/>
Deaths - Cumulative number of of deaths till that date<br/>
Recovered - Cumulative number of recovered cases till that date

### 2. COVID_open_line_list_data.csv and COVID19_line_list_data.csv

Individual level data information
<br/><br/>

ID<br/>
age<br/>
sex<br/>
city<br/>
province<br/>
country<br/>
wuhan(0)_not_wuhan(1)<br/>
latitude<br/>
longitude<br/>
geo_resolution<br/>
date_onset_symptoms<br/>
date_admission_hospital<br/>
date_confirmation<br/>
symptoms<br/>
lives_in_Wuhan<br/>
travel_history_dates<br/>
travel_history_location<br/>
reported_market_exposure<br/>
additional_information<br/>
chronic_disease_binary<br/>
chronic_disease<br/>
source<br/>
sequence_available<br/>
outcome<br/>
date_death_or_discharge<br/>
notes_for_discussion<br/>
location<br/>
admin3<br/>
admin2<br/>
admin1<br/>
country_new<br/>
admin_id<br/>
data_moderator_initials<br/>

### Region wise data

In [7]:
region_wise_data = drop_null_vals(pd.read_csv('data/covid_19_data.csv'),axis="both")
region_wise_data

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
6157,6158,03/16/2020,Cayman Islands,UK,2020-03-16T14:53:04,1.0,1.0,0.0
6158,6159,03/16/2020,Gibraltar,UK,2020-03-14T16:33:03,1.0,0.0,1.0
6159,6160,03/16/2020,Diamond Princess cruise ship,Australia,2020-03-14T02:33:04,0.0,0.0,0.0
6160,6161,03/16/2020,West Virginia,US,2020-03-10T02:33:04,0.0,0.0,0.0


### Individual information

In [8]:
open_line_list = drop_null_vals(pd.read_csv('data/COVID19_open_line_list.csv'),axis='both')
open_line_list

,ID,age,sex,city,province,country,wuhan(0)_not_wuhan(1),latitude,longitude,geo_resolution,...,outcome,date_death_or_discharge,notes_for_discussion,location,admin3,admin2,admin1,country_new,admin_id,data_moderator_initials
0,1.0,30,male,"Chaohu City, Hefei City",Anhui,China,1.0,31.646960,117.716600,admin3,...,NaN,NaN,NaN,NaN,Chaohu City,Hefei City,Anhui,China,340181,NaN
1,2.0,47,male,"Baohe District, Hefei City",Anhui,China,1.0,31.778630,117.331900,admin3,...,NaN,NaN,NaN,NaN,Baohe District,Hefei City,Anhui,China,340111,NaN
2,3.0,49,male,"High-Tech Zone, Hefei City",Anhui,China,1.0,31.828313,117.224844,point,...,NaN,NaN,NaN,High-Tech Zone,Shushan District,Hefei City,Anhui,China,340104,NaN
3,4.0,47,female,"High-Tech Zone, Hefei City",Anhui,China,1.0,31.828313,117.224844,point,...,NaN,NaN,NaN,High-Tech Zone,Shushan District,Hefei City,Anhui,China,340104,NaN
4,5.0,50,female,"Feidong County, Hefei City",Anhui,China,1.0,32.001230,117.568100,admin3,...,NaN,NaN,NaN,NaN,Feidong County,Hefei City,Anhui,China,340122,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13169,13475.0,60-69,male,Nagoya City,Aichi Prefecture,Japan,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13170,13476.0,40-49,female,NaN,Osaka,Japan,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13171,13477.0,0-10,female,NaN,Osaka,Japan,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13172,13478.0,NaN,female,Amsterdam,NaN,Netherlands,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
line_list = drop_null_vals(pd.read_csv('data/COVID19_line_list_data.csv'),'both')
line_list

,id,case_in_country,reporting date,summary,location,country,gender,age,symptom_onset,If_onset_approximated,hosp_visit_date,exposure_start,exposure_end,visiting Wuhan,from Wuhan,death,recovered,symptom,source,link
0,1,NaN,1/20/2020,First confirmed imported COVID-19 pneumonia pa...,"Shenzhen, Guangdong",China,male,66.0,01/03/20,0.0,01/11/20,12/29/2019,01/04/20,1,0.0,0,0,NaN,Shenzhen Municipal Health Commission,http://wjw.sz.gov.cn/wzx/202001/t20200120_1898...
1,2,NaN,1/20/2020,First confirmed imported COVID-19 pneumonia pa...,Shanghai,China,female,56.0,1/15/2020,0.0,1/15/2020,NaN,01/12/20,0,1.0,0,0,NaN,Official Weibo of Shanghai Municipal Health Co...,https://www.weibo.com/2372649470/IqogQhgfa?fro...
2,3,NaN,1/21/2020,First confirmed imported cases in Zhejiang: pa...,Zhejiang,China,male,46.0,01/04/20,0.0,1/17/2020,NaN,01/03/20,0,1.0,0,0,NaN,Health Commission of Zhejiang Province,http://www.zjwjw.gov.cn/art/2020/1/21/art_1202...
3,4,NaN,1/21/2020,new confirmed imported COVID-19 pneumonia in T...,Tianjin,China,female,60.0,NaN,NaN,1/19/2020,NaN,NaN,1,0.0,0,0,NaN,人民日报官方微博,https://m.weibo.cn/status/4463235401268457?
4,5,NaN,1/21/2020,new confirmed imported COVID-19 pneumonia in T...,Tianjin,China,male,58.0,NaN,NaN,1/14/2020,NaN,NaN,0,0.0,0,0,NaN,人民日报官方微博,https://m.weibo.cn/status/4463235401268457?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080,1081,2.0,2/25/2020,"new COVID-19 patient confirmed in Austria: 24,...",Innsbruck,Austria,NaN,24.0,NaN,NaN,NaN,NaN,NaN,0,0.0,0,0,NaN,Jakarta Post,https://www.thejakartapost.com/news/2020/02/25...
1081,1082,1.0,2/24/2020,new COVID-19 patient confirmed in Afghanistan:...,Afghanistan,Afghanistan,NaN,35.0,NaN,NaN,NaN,NaN,NaN,0,0.0,0,0,NaN,Fox News,https://www.foxnews.com/health/afghanistan-fir...
1082,1083,1.0,2/26/2020,new COVID-19 patient confirmed in Algeria: mal...,Algeria,Algeria,male,NaN,NaN,NaN,NaN,NaN,2/17/2020,0,0.0,0,0,NaN,South China Morning Post,https://www.scmp.com/news/china/society/articl...
1083,1084,1.0,2/25/2020,new COVID-19 patient confirmed in Croatia: mal...,Croatia,Croatia,male,NaN,NaN,NaN,NaN,2/19/2020,2/21/2020,0,0.0,0,0,NaN,Reuters,https://www.reuters.com/article/us-croatia-cor...


### Time series data (John Hopkins)

In [10]:
time_series_confirmed = drop_null_vals(pd.read_csv('data/time_series_covid_19_confirmed.csv'))
time_series_deaths = drop_null_vals(pd.read_csv('data/time_series_covid_19_deaths.csv'))
time_series_recovered = drop_null_vals(pd.read_csv('data/time_series_covid_19_recovered.csv'))


In [11]:
series_confirmed_coord = drop_null_vals(time_series_confirmed,subset=['Lat','Long'])
series_confirmed_coord

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,47,48,50,50,50,53,59,70,75,82
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,360,420,461,502,511,581,639,639,701,773
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,117,130,138,150,150,160,178,178,200,212
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,50,83,93,99,117,129,149,149,197,238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,NaN,Saint Lucia,13.9094,-60.9789,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
438,NaN,Saint Vincent and the Grenadines,12.9843,-61.2872,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
439,NaN,Suriname,3.9193,-56.0278,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
440,"Virgin Islands, U.S.",US,18.3358,-64.8963,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### Temperature and humidity data

In [12]:
lat_long_list = list(series_confirmed_coord[['Lat','Long']].values)

<b><u>API Sample Calls</u></b>

API to fetch station: https://api.meteostat.net/v1/stations/nearby?lat=1.283&lon=103.83&limit=1&key=XXXXXXX<br/>
API to fetch daily historical data(not used any more): https://api.meteostat.net/v1/history/daily?station=10637&start=2017-01-01&end=2017-12-31&key=XXXXXXXX<br/>
API to fetch historical hourly data: https://api.meteostat.net/v1/history/hourly?station=03772&start=2019-05-02&end=2019-05-11&time_zone=Europe/London&time_format=Y-m-d%20H:i&key=XXXXXXXX


In [13]:
START_DATE = str(datetime.strptime(series_confirmed_coord.columns[4], '%m/%d/%y').date())
END_DATE = str(datetime.strptime(series_confirmed_coord.columns[-1], '%m/%d/%y').date())
complete_weather_data = dict()

In [24]:
DATA

{'15.0101.0':              2020-01-22  2020-01-23  2020-01-24  2020-01-25  2020-01-26  \
 temperature        30.8       31.06        29.3       29.34       30.38   
 humidity           72.0       72.80        75.8       70.80       67.20   
 
              2020-01-27  2020-01-28  2020-01-29  2020-01-30  2020-01-31  ...  \
 temperature       31.16       29.82       29.76       29.58       29.74  ...   
 humidity          56.60       60.80       64.20       68.40       53.80  ...   
 
              2020-03-05  2020-03-06  2020-03-07  2020-03-08  2020-03-09  \
 temperature        30.7       29.56        29.8        29.9        30.5   
 humidity           69.2       73.40        74.6        72.4        68.6   
 
              2020-03-10  2020-03-11  2020-03-12  2020-03-13  2020-03-14  
 temperature       30.76        30.8       30.56   30.500000      31.125  
 humidity          68.80        69.8       71.80   76.666667      73.500  
 
 [2 rows x 53 columns], '36.0138.0': Empty DataFrame
 C

In [23]:
for lat_long in lat_long_list:
    if(str(lat_long[0])+str(lat_long[1]) not in complete_weather_data):
        complete_weather_data[str(lat_long[0])+str(lat_long[1])] = getWeatherData(lat_long[0],lat_long[1])
        time.sleep(60)

complete_weather_data

{'15.0101.0':              2020-01-22  2020-01-23  2020-01-24  2020-01-25  2020-01-26  \
 temperature        30.8       31.06        29.3       29.34       30.38   
 humidity           72.0       72.80        75.8       70.80       67.20   
 
              2020-01-27  2020-01-28  2020-01-29  2020-01-30  2020-01-31  ...  \
 temperature       31.16       29.82       29.76       29.58       29.74  ...   
 humidity          56.60       60.80       64.20       68.40       53.80  ...   
 
              2020-03-05  2020-03-06  2020-03-07  2020-03-08  2020-03-09  \
 temperature        30.7       29.56        29.8        29.9        30.5   
 humidity           69.2       73.40        74.6        72.4        68.6   
 
              2020-03-10  2020-03-11  2020-03-12  2020-03-13  2020-03-14  
 temperature       30.76        30.8       30.56   30.500000      31.125  
 humidity          68.80        69.8       71.80   76.666667      73.500  
 
 [2 rows x 53 columns], '36.0138.0': Empty DataFrame
 C

In [246]:
def getCompleteWeatherData(coordinate_list,complete_weather_data=dict(),flag=True):
    '''
    Returns consolidated weather
    data for all the coordinates
    in list of coordinates.
    
    Parameters
    ----------
    coordinate_list : list(tuple)
        List of coordinates to
        be evaluated.
        
    complete_weather_data : dict
        Map of coordinate to
        weather data.
    
    flag : bool
        If True, means some
       coordinate_listts failed, retry
        the failed requests.
    '''
    assert(isinstance(flag,bool))
    assert(isinstance(complete_weather_data,dict))
    assert(isinstance(coordinate_list,list))
    
    try:
        if(flag==True):
            flag=False
            for lat_long in lat_long_list:
                coordinate = (str(lat_long[0]),str(lat_long[1]))
                if((coordinate not in complete_weather_data) and (coordinate not in blacklist)):
                    flag = True
                    complete_weather_data[(str(lat_long[0]),str(lat_long[1]))] = getWeatherData(lat_long[0],lat_long[1])
                    time.sleep(10)
            return getCompleteWeatherData(coordinate_list,complete_weather_data,flag)
        else:
            return complete_weather_data
    except Exception as ex:
        print(ex)
        return complete_weather_data

In [247]:
temp = getCompleteWeatherData(lat_long_list)
STATIONS.to_csv(r'data\stations.csv', index=False)

Station does not exist in "Stations.csv", fetching from api...


C:\Users\singh\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


Station fetched for (15.000000,101.000000) is: 48463
Station found for (15.000000,101.000000):
48463
URL for getting weather data for (15.000000,101.000000):
https://api.meteostat.net/v1/history/hourly?station=48463&start=2020-01-22&end=2020-03-14&time_zone=Europe/London&time_format=Y-m-d&key=VJga0fAn
Got response, status = 200.000000
Station does not exist in "Stations.csv", fetching from api...
Station fetched for (36.000000,138.000000) is: 47620
Station found for (36.000000,138.000000):
47620
URL for getting weather data for (36.000000,138.000000):
https://api.meteostat.net/v1/history/hourly?station=47620&start=2020-01-22&end=2020-03-14&time_zone=Europe/London&time_format=Y-m-d&key=VJga0fAn
Got response, status = 200.000000
No data found for (36.000000,138.000000)
Retry has to be done for (36.000000,138.000000)
Station does not exist in "Stations.csv", fetching from api...
Station fetched for (1.283300,103.833300) is: 48694
Station found for (1.283300,103.833300):
48694
URL for gett

Got response, status = 200.000000
No data found for (-34.928500,138.600700)
Retry has to be done for (-34.928500,138.600700)
Station does not exist in "Stations.csv", fetching from api...
Station fetched for (50.833300,4.000000) is: 06434
Station found for (50.833300,4.000000):
06434
URL for getting weather data for (50.833300,4.000000):
https://api.meteostat.net/v1/history/hourly?station=06434&start=2020-01-22&end=2020-03-14&time_zone=Europe/London&time_format=Y-m-d&key=VJga0fAn
Got response, status = 200.000000
Station does not exist in "Stations.csv", fetching from api...
No station found for (26.000000,30.000000). URL: https://api.meteostat.net/v1/stations/nearby?lat=26.0&lon=30.0&limit=1&key=VJga0fAn
exception:  list index out of range
Station does not exist in "Stations.csv", fetching from api...
Station fetched for (35.443700,139.638000) is: 47670
Station found for (35.443700,139.638000):
47670
URL for getting weather data for (35.443700,139.638000):
https://api.meteostat.net/v1

Got response, status = 200.000000
Station does not exist in "Stations.csv", fetching from api...
Station fetched for (58.595300,25.013600) is: 26135
Station found for (58.595300,25.013600):
26135
URL for getting weather data for (58.595300,25.013600):
https://api.meteostat.net/v1/history/hourly?station=26135&start=2020-01-22&end=2020-03-14&time_zone=Europe/London&time_format=Y-m-d&key=VJga0fAn
Got response, status = 200.000000
No data found for (58.595300,25.013600)
Retry has to be done for (58.595300,25.013600)
Station does not exist in "Stations.csv", fetching from api...
Station fetched for (52.132600,5.291300) is: 06265
Station found for (52.132600,5.291300):
06265
URL for getting weather data for (52.132600,5.291300):
https://api.meteostat.net/v1/history/hourly?station=06265&start=2020-01-22&end=2020-03-14&time_zone=Europe/London&time_format=Y-m-d&key=VJga0fAn
Got response, status = 200.000000
No data found for (52.132600,5.291300)
Retry has to be done for (52.132600,5.291300)
Sta

...

Station fetched for (27.990400,-82.301800) is: KVDF0
Station found for (27.990400,-82.301800):
KVDF0
URL for getting weather data for (27.990400,-82.301800):
https://api.meteostat.net/v1/history/hourly?station=KVDF0&start=2020-01-22&end=2020-03-14&time_zone=Europe/London&time_format=Y-m-d&key=VJga0fAn
Got response, status = 200.000000
Station does not exist in "Stations.csv", fetching from api...
Station fetched for (39.091600,-120.803900) is: KBLU0
Station found for (39.091600,-120.803900):
KBLU0
URL for getting weather data for (39.091600,-120.803900):
https://api.meteostat.net/v1/history/hourly?station=KBLU0&start=2020-01-22&end=2020-03-14&time_zone=Europe/London&time_format=Y-m-d&key=VJga0fAn
Got response, status = 403.000000
No data found for (39.091600,-120.803900)
Retry has to be done for (39.091600,-120.803900)
Station does not exist in "Stations.csv", fetching from api...
Station fetched for (37.563000,-122.325500) is: 72494
Station found for (37.563000,-122.325500):
72494
URL

Station fetched for (38.083400,-122.763300) is: KDVO0
Station found for (38.083400,-122.763300):
KDVO0
URL for getting weather data for (38.083400,-122.763300):
https://api.meteostat.net/v1/history/hourly?station=KDVO0&start=2020-01-22&end=2020-03-14&time_zone=Europe/London&time_format=Y-m-d&key=VJga0fAn
Got response, status = 200.000000
Station does not exist in "Stations.csv", fetching from api...
Station fetched for (38.196000,-120.680500) is: KCPU0
Station found for (38.196000,-120.680500):
KCPU0
URL for getting weather data for (38.196000,-120.680500):
https://api.meteostat.net/v1/history/hourly?station=KCPU0&start=2020-01-22&end=2020-03-14&time_zone=Europe/London&time_format=Y-m-d&key=VJga0fAn
Got response, status = 200.000000
Station does not exist in "Stations.csv", fetching from api...
Station fetched for (37.509100,-120.987600) is: KMOD0
Station found for (37.509100,-120.987600):
KMOD0
URL for getting weather data for (37.509100,-120.987600):
https://api.meteostat.net/v1/hist

Got response, status = 200.000000
Station does not exist in "Stations.csv", fetching from api...
Station fetched for (40.410800,-75.247900) is: KCKZ0
Station found for (40.410800,-75.247900):
KCKZ0
URL for getting weather data for (40.410800,-75.247900):
https://api.meteostat.net/v1/history/hourly?station=KCKZ0&start=2020-01-22&end=2020-03-14&time_zone=Europe/London&time_format=Y-m-d&key=VJga0fAn
Got response, status = 200.000000
Station does not exist in "Stations.csv", fetching from api...
Station fetched for (37.777200,-77.516100) is: KOFP0
Station found for (37.777200,-77.516100):
KOFP0
URL for getting weather data for (37.777200,-77.516100):
https://api.meteostat.net/v1/history/hourly?station=KOFP0&start=2020-01-22&end=2020-03-14&time_zone=Europe/London&time_format=Y-m-d&key=VJga0fAn
Got response, status = 200.000000
Station does not exist in "Stations.csv", fetching from api...
Station fetched for (34.725300,-80.677100) is: KLKR0
Station found for (34.725300,-80.677100):
KLKR0
UR

Got response, status = 200.000000
Station does not exist in "Stations.csv", fetching from api...
Station fetched for (42.981500,-97.872200) is: KYKN0
Station found for (42.981500,-97.872200):
KYKN0
URL for getting weather data for (42.981500,-97.872200):
https://api.meteostat.net/v1/history/hourly?station=KYKN0&start=2020-01-22&end=2020-03-14&time_zone=Europe/London&time_format=Y-m-d&key=VJga0fAn
Got response, status = 200.000000
Station does not exist in "Stations.csv", fetching from api...
Station fetched for (33.883700,-106.723500) is: 72362
Station found for (33.883700,-106.723500):
72362
URL for getting weather data for (33.883700,-106.723500):
https://api.meteostat.net/v1/history/hourly?station=72362&start=2020-01-22&end=2020-03-14&time_zone=Europe/London&time_format=Y-m-d&key=VJga0fAn
Got response, status = 200.000000
Station does not exist in "Stations.csv", fetching from api...
Station fetched for (35.017800,-106.629100) is: 72365
Station found for (35.017800,-106.629100):
723

Station does not exist in "Stations.csv", fetching from api...
No station found for (17.060800,-61.796400). URL: https://api.meteostat.net/v1/stations/nearby?lat=17.0608&lon=-61.7964&limit=1&key=VJga0fAn
exception:  Expecting value: line 1 column 1 (char 0)
Retry has to be done for (17.060800,-61.796400)
Station does not exist in "Stations.csv", fetching from api...
No station found for (32.318200,-86.902300). URL: https://api.meteostat.net/v1/stations/nearby?lat=32.3182&lon=-86.9023&limit=1&key=VJga0fAn
exception:  Expecting value: line 1 column 1 (char 0)
Retry has to be done for (32.318200,-86.902300)
Station does not exist in "Stations.csv", fetching from api...
No station found for (4.000000,-53.000000). URL: https://api.meteostat.net/v1/stations/nearby?lat=4.0&lon=-53.0&limit=1&key=VJga0fAn
exception:  Expecting value: line 1 column 1 (char 0)
Retry has to be done for (4.000000,-53.000000)
Station does not exist in "Stations.csv", fetching from api...
No station found for (-32.52

In [262]:
pd.DataFrame(temp).dropna(how="all",axis=1)

,15.0,1.2833,28.1667,2.5,49.2827,-33.8688,-37.8136,11.55,51.0,64.0,...,35.0178,42.5922,42.2791,39.5393,22.0,55.3781,19.3133,16.265,53.7609,12.8628
,101.0,103.8333,84.25,112.5,-123.1207,151.2093,144.9631,104.9167,9.0,26.0,...,-106.6291,-83.3362,-83.3362,-75.6674,-80.0,-3.4360000000000004,-81.2546,-61.551,-98.8139,30.2176
2020-01-22,"{'temperature': 30.8, 'humidity': 72.0}","{'temperature': 28.833333333333332, 'humidity'...","{'temperature': 10.870833333333332, 'humidity'...","{'temperature': 27.241666666666664, 'humidity'...","{'temperature': 6.720833333333334, 'humidity':...","{'temperature': 26.308333333333337, 'humidity'...","{'temperature': 21.3875, 'humidity': 62.416666...","{'temperature': 29.208333333333332, 'humidity'...","{'temperature': -2.4041666666666672, 'humidity...","{'temperature': -2.754166666666667, 'humidity'...",...,"{'temperature': 4.808333333333332, 'humidity':...","{'temperature': -4.954166666666667, 'humidity'...","{'temperature': -4.574999999999999, 'humidity'...","{'temperature': -1.8333333333333333, 'humidity...","{'temperature': 17.57916666666667, 'humidity':...","{'temperature': 7.954166666666667, 'humidity':...","{'temperature': 22.735294117647058, 'humidity'...","{'temperature': 23.650000000000002, 'humidity'...","{'temperature': -7.470833333333331, 'humidity'...","{'temperature': 18.704166666666666, 'humidity'..."
2020-01-23,"{'temperature': 31.060000000000002, 'humidity'...","{'temperature': 28.625, 'humidity': 76.9583333...","{'temperature': 11.312500000000002, 'humidity'...","{'temperature': 27.3625, 'humidity': 88.0}","{'temperature': 7.020833333333333, 'humidity':...","{'temperature': 29.21666666666667, 'humidity':...","{'temperature': 16.620833333333334, 'humidity'...","{'temperature': 28.833333333333332, 'humidity'...","{'temperature': -0.7041666666666667, 'humidity...","{'temperature': -3.1666666666666674, 'humidity...",...,"{'temperature': 5.208333333333333, 'humidity':...","{'temperature': -1.1749999999999998, 'humidity...","{'temperature': -1.1124999999999998, 'humidity...","{'temperature': -0.08333333333333333, 'humidit...","{'temperature': 17.40833333333334, 'humidity':...","{'temperature': 5.675000000000001, 'humidity':...","{'temperature': 22.41176470588235, 'humidity':...","{'temperature': 25.287499999999998, 'humidity'...","{'temperature': -19.22083333333333, 'humidity'...","{'temperature': 17.495833333333334, 'humidity'..."
2020-01-24,"{'temperature': 29.3, 'humidity': 75.8}","{'temperature': 28.125, 'humidity': 74.3333333...","{'temperature': 10.558333333333332, 'humidity'...","{'temperature': 26.137500000000003, 'humidity'...","{'temperature': 8.808333333333332, 'humidity':...","{'temperature': 24.42083333333333, 'humidity':...","{'temperature': 19.591666666666665, 'humidity'...","{'temperature': 28.583333333333332, 'humidity'...","{'temperature': -1.575, 'humidity': 99.875}","{'temperature': -0.9333333333333332, 'humidity...",...,"{'temperature': 4.8875, 'humidity': 59.7083333...","{'temperature': 1.133333333333333, 'humidity':...","{'temperature': 2.720833333333332, 'humidity':...","{'temperature': 4.25, 'humidity': 64.083333333...","{'temperature': 20.150000000000002, 'humidity'...","{'temperature': 5.958333333333335, 'humidity':...","{'temperature': 24.41176470588235, 'humidity':...","{'temperature': 24.270833333333332, 'humidity'...","{'temperature': -11.241666666666665, 'humidity...","{'temperature': 17.433333333333334, 'humidity'..."
2020-01-25,"{'temperature': 29.340000000000003, 'humidity'...","{'temperature': 28.416666666666668, 'humidity'...","{'temperature': 10.754166666666668, 'humidity'...","{'temperature': 27.820833333333336, 'humidity'...","{'temperature': 7.433333333333334, 'humidity':...","{'temperature': 25.412499999999994, 'humidity'...","{'temperature': 20.004166666666663, 'humidity'...","{'temperature': 28.5, 'humidity': 68.083333333...","{'temperature': -1.6375, 'humidity': 100.0}","{'temperature': -4.654166666666667, 'humidity'...",...,"{'temperature': 

In [294]:
new_dict = {}
for key in list(temp.keys()):
    new_dict[str(key[0])+","+str(key[1])] = temp[key]
    
my_dict = pd.DataFrame(new_dict).dropna(how="all",axis=1).to_dict()
with open('data/weather_mapping_data.json', 'w') as json_file:
    json.dump(my_dict, json_file)